In [1]:
import json
from paperqa import Settings, ask
import os
from paperqa import Settings, Docs
from paperqa.agents.main import agent_query
from paperqa.agents.search import get_directory_index

from aviary.env import TaskDataset
from ldp.agent import SimpleAgent
from ldp.alg.callbacks import MeanMetricsCallback
from ldp.alg.runners import Evaluator, EvaluatorConfig
import sys

In [2]:
sys.path.append("src")

In [3]:
from MC_Grader import MultipleChoiceGrader, process_results, calculate_metrics_from_file
from config import OPENAI_API_KEY
from openai import OpenAI

## Load the Question 

In [ ]:
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/questions/formatted_questions_test/questions.json", "r") as f:
    questions = json.load(f)

In [5]:
print(questions)

[{'question': 'Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?', 'choices': ['(A) 41%', '(B) 51%', '(C) 31%', '(D) Insufficient information to answer the question', '(E) 11%', '(F) 21%'], 'correct_answer': 'C', 'unsure_option': 'D', 'sources': ['https://doi.org/10.1038/s41467-024-44782-6'], 'ideal': '31%', 'distractors': ['21%', '11%', '41%', '51%']}, {'question': 'At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?', 'choices': ['(A) 0 h', '(B) 6 h', '(C) 24', '(D) 12 h', '(E) 48 h', '(F) 96 h', '(G) Insufficient information to answer the question', '(H) 72 h'], 'correct_answer': 'H', 'unsure_option': 'G', 'sources': ['https://doi.org/10.1073/pnas.2322688121'], 'ideal': '72 h', 'distractors': ['24', '48 h', '0 h', '12 h', '6 h', '96 h']}, {'question': 'DK015 and DK038 strains of Verticillium dahliae have in common approximately what 

In [6]:
def format_multiple_choice_question(question, choices):
    formatted = f"Question:Q: {question}; Options: "
    formatted += " ".join(choices)
    return formatted


In [7]:
formatted_q=[]

In [8]:
for q in questions:
    formatted_q.append(format_multiple_choice_question(q["question"], q["choices"]))

In [9]:
len(formatted_q)

49

In [10]:
print(formatted_q[:5])

['Question:Q: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%', 'Question:Q: At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?; Options: (A) 0 h (B) 6 h (C) 24 (D) 12 h (E) 48 h (F) 96 h (G) Insufficient information to answer the question (H) 72 h', 'Question:Q: DK015 and DK038 strains of Verticillium dahliae have in common approximately what percentage orthologous genes?; Options: (A) 97% (B) Insufficient information to answer the question (C) 95% (D) 98% (E) 96% (F) 94%', 'Question:Q: Expression of which of the following genes was significantly reduced in the cortical plate of olig2-/- mice compared to WT mice at E18.5?; Options: (A) none of the above (B) Insufficient information to answer the question (C) Aldh1l1 (D) Actin (E) MA

## Get the Paperqa Response

In [11]:
#build the index
# Create settings with explicit configuration
settings = Settings(
    paper_directory='/Users/apple/Documents/GitLab_Projects/master_project/xx823/papers',
    agent={"index": {
        "sync_with_paper_directory": True,
        "recurse_subdirectories": True
    }}
)

built_index = await get_directory_index(settings=settings)

# Print index information 
print(f"Using index: {settings.get_index_name()}")
index_files = await built_index.index_files
print(f"Number of indexed files: {len(index_files)}")
print("Indexed files:")
for file in index_files:
    print(f"- {file}")

Using index: pqa_index_73db35b7edb188828b5799b8ac0a3fac
Number of indexed files: 49
Indexed files:
- DiffDomain enables identification of structurally reorganized topologically associating domains.pdf
- Full-length tRNAs lacking a functional CCA tail are selectively sorted into the lumen of extracellular vesicles.pdf
- Type-I-interferon-responsive microglia shape cortical development and behavior.pdf
- liang-et-al-2024-a-conserved-molecular-logic-for-neurogenesis-to-gliogenesis-switch-in-the-cerebral-cortex.pdf
- goyette-et-al-2024-cancer-stromal-cell-interactions-in-breast-cancer-brain-metastases-induce-glycocalyx-mediated.pdf
- Light regulates widespread plant alternative polyadenylation 2 through the chloroplast.pdf
- High-speed imaging of giant unilamellar vesicle formation in cDICE.pdf
- elife-90425-v2.pdf
- sherman-et-al-2024-the-fatty-liver-disease-causing-protein-pnpla3-i148m-alters-lipid-droplet-golgi-dynamics.pdf
- Role of m6A RNA methylation in dosage compensation.pdf
- Func

In [12]:
def get_paperqa2_response(question_data):
    """
    This function should call PaperQA2 with the question and return the response.
    Adjust this function based on how you're currently calling PaperQA2.
    """
    # Load the settings
    settings = Settings(
    temperature=0.5,  # Keep deterministic
    paper_directory="/Users/apple/Documents/GitLab_Projects/master_project/xx823/papers"  # Directory containing the paper with DOI: 10.1101/2024.01.31.578101
)

    # Format the question as needed for PaperQA2
    #format_multiple_choice_question(question_data['question'], question_data['choices'])

    import nest_asyncio
    nest_asyncio.apply()
    # Call PaperQA2 with the question
    response_pqa = ask(question_data, settings=settings)
    answer_dict=response_pqa.dict()

    return answer_dict['session']['answer']


## Create the required data structure

In [13]:
grader_input_data = []

In [57]:
formatted_q[0]

'Question:Q: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%'

In [14]:
total_questions = len(formatted_q)
for i, q in enumerate(formatted_q): # test for 49 questions
    current_question_num = i + 1
    print(f"Processing question {current_question_num}/{total_questions}: {q[:50]}...")
    
    # Get PaperQA2's response for this question
    response = get_paperqa2_response(q)
    
    # Log completion status
    print(f"Completed question {current_question_num}/{total_questions}, {total_questions - current_question_num} remaining")
    
    # Add the question and response to the grader input
    grader_input_data.append({
        "question":questions[i]["question"],
        "choices": questions[i]["choices"],
        "response": response,
        "correct_answer": questions[i]["correct_answer"],
        "unsure_option": questions[i]["unsure_option"]
    })

print(f"All {total_questions} questions processed successfully")


Processing question 1/49: Question:Q: Approximately what percentage of topol...


[12:02:36] Starting paper search for 'DiffDomain classify topologically associated domains GM12878 K562'.

2025-03-01 12:02:36,876 - paperqa.agents.tools - INFO - Starting paper search for 'DiffDomain classify topologically associated domains GM12878 K562'.


           paper_search for query 'DiffDomain classify topologically associated domains GM12878 K562' and offset 0 
           returned 8 papers.

2025-03-01 12:02:36,962 - paperqa.agents.tools - INFO - paper_search for query 'DiffDomain classify topologically associated domains GM12878 K562' and offset 0 returned 8 papers.


...

[12:17:50] Completing 'Question:Q: dimt-1 knockdown in C. elegans has been shown to cause upregulation of genes in 
           which of the following pathways?; Options: (A) FOXO signaling (B) WNT signaling (C) mTOR signaling (D)  
           Insufficient information to answer the question (E) ribosome biogenesis' as 'certain'.

2025-03-01 12:17:50,388 - paperqa.agents.tools - INFO - Completing 'Question:Q: dimt-1 knockdown in C. elegans has been shown to cause upregulation of genes in which of the following pathways?; Options: (A) FOXO signaling (B) WNT signaling (C) mTOR signaling (D) Insufficient information to answer the question (E) ribosome biogenesis' as 'certain'.


           Answer: Dimt-1 knockdown in *Caenorhabditis elegans* has been shown to cause the upregulation of genes  
           involved in several pathways, including WNT signaling. Pathway analysis of transcriptional changes      
           following dimt-1 knockdown revealed significant upregulation of 3,700 genes, with enrichment in pathways
           such as WNT signaling, TGF-β signaling, MAPK signaling, and degradation pathways like proteasome,       
           peroxisome, and autophagy (rothi2024the18srrna pages 5-7). Additionally, dimt-1 depletion altered       
           ribosomal binding to mRNAs involved in longevity regulation, degradation pathways, oxidative            
           phosphorylation, and glutathione metabolism (rothi2024the18srrna pages 7-8).                            
                                                                                                                   
           However, there is no evidence in the provided context indicating the upregulation of genes in the FOXO  
           signaling or mTOR signaling pathways. While dimt-1 knockdown influences lifespan through the FOXO       
           pathway, as indicated by its dependency on daf-16 for lifespan extension, this does not confirm         
           transcriptional upregulation of FOXO pathway genes (rothi2024the18srrna pages 17-18). Similarly, the    
           involvement of the TOR pathway in lifespan regulation is noted, but no evidence suggests upregulation of
           TOR pathway genes (rothi2024the18srrna pages 17-18).                                                    
                                                                                                                   
           Thus, among the provided options, dimt-1 knockdown upregulates genes in the WNT signaling pathway.

2025-03-01 12:17:50,528 - paperqa.agents.main.agent_callers - INFO - [bold blue]Answer: Dimt-1 knockdown in *Caenorhabditis elegans* has been shown to cause the upregulation of genes involved in several pathways, including WNT signaling. Pathway analysis of transcriptional changes following dimt-1 knockdown revealed significant upregulation of 3,700 genes, with enrichment in pathways such as WNT signaling, TGF-β signaling, MAPK signaling, and degradation pathways like proteasome, peroxisome, and autophagy (rothi2024the18srrna pages 5-7). Additionally, dimt-1 depletion altered ribosomal binding to mRNAs involved in longevity regulation, degradation pathways, oxidative phosphorylation, and glutathione metabolism (rothi2024the18srrna pages 7-8). 

However, there is no evidence in the provided context indicating the upregulation of genes in the FOXO signaling or mTOR signaling pathways. While dimt-1 knockdown influences lifespan through the FOXO pathway, as indicated by its dependency on d

Completed question 49/49, 0 remaining
All 49 questions processed successfully


In [ ]:
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/paperqa_response/grader_input_data_test.json", "w") as f:
    json.dump(grader_input_data, f, indent=2)

In [ ]:
#load the previous data if needed
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/paperqa_response/grader_input_data_test.json", 'r') as f:
        grader_input_data = json.load(f)
    

In [5]:
grader = MultipleChoiceGrader(
    # Add your OpenAI API key here if not set in environment
    openai_api_key=OPENAI_API_KEY,
    results_dir="grading_results_49MCQs_test"
)


In [6]:
# Use the fixed version instead of the original
process_results(grader, grader_input_data, save_prefix="paperqa2_evaluation")

2025-03-01 12:25:34,679 - MC_Grader - INFO - Processing question 1/49
2025-03-01 12:25:34,680 - MC_Grader - INFO - Processing question: Approximately what percentage of topologically ass...


HumanProxy (to GraderAgent):


    Question:
    Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?

    Options:
    (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%

    PaperQA2 Response:
    DiffDomain classifies approximately 30.771% of topologically associated domains (TADs) in the GM12878 blood cell line as reorganized in the K562 cell line. This percentage is significantly higher than those identified by other methods, such as TADCompare, HiCcompare, and HiC-DC+, which detect reorganized TADs at rates of ≤8.256% (hua2024diffdomainenablesidentification pages 4-4). Additionally, DiffDomain demonstrates substantial improvements in identifying reorganized TADs, with a minimum of 43.137%, median of 81.357%, and maximum of 98.022% of reorganized GM12878 TADs being identified in the K562 cell line, depending on specific thresholds and 

2025-03-01 12:25:35,296 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


GraderAgent (to HumanProxy):

C

--------------------------------------------------------------------------------


2025-03-01 12:25:36,324 - MC_Grader - INFO - Processing question 2/49
2025-03-01 12:25:36,326 - MC_Grader - INFO - Processing question: At least how long do SynNotch-MCF10DCIS cells expr...


...

2025-03-01 12:26:58,120 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


GraderAgent (to HumanProxy):

B

--------------------------------------------------------------------------------


2025-03-01 12:26:59,155 - MC_Grader - INFO - Results saved to grading_results_49MCQs_test



Results saved to grading_results_49MCQs_test
- CSV: paperqa2_evaluation_extracted_answers.csv
- JSON: paperqa2_evaluation_extraction_results.json

You can now manually verify the results and calculate metrics.


'grading_results_49MCQs_test/paperqa2_evaluation_extraction_results.json'

In [ ]:
metrics = calculate_metrics_from_file("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/grading_results_49MCQs_test_paperqa/paperqa2_evaluation_extraction_results.json")


Metrics:
Total questions: 49
Correct answers: 39 (79.59%)
Incorrect answers: 6 (12.24%)
Unsure answers: 4 (8.16%)
Precision: 86.67%
Accuracy: 79.59%
Metrics saved to /Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/grading_results_49MCQs_test/metrics.csv
